In [2]:
import os
from collections import OrderedDict
from torch.autograd import Variable
from options.test_options import TestOptions
from data.ov_test_dataset import TestDataset
from models.models import create_model
import util.util as util
from util.visualizer import Visualizer
from util import html
import torch
from torchvision import transforms
from torch.utils.data import DataLoader

opt = TestOptions().parse(save=False)
opt.nThreads = 1   # test code only supports nThreads = 1
opt.batchSize = 1  # test code only supports batchSize = 1
opt.serial_batches = True  # no shuffle
opt.no_flip = True  # no flip

opt.label_feat = True
opt.model = 'ov_pix2pixHD'
opt.name = 'vfr_shape'

opt.shape_generation = True
# opt.use_generator_last_activation = False
opt.input_nc= 20 
opt.output_nc = 20

opt.dataroot= '/workspace/home/aden/work/project/o-viton/dataset/'
opt.checkpoints_dir = "/workspace/home/aden/work/project/o-viton/checkpoints"
opt.results_dir  = "/workspace/home/aden/work/project/o-viton/results"

opt

------------ Options -------------
appearance_generation: False
aspect_ratio: 1.0
batchSize: 1
checkpoints_dir: ./checkpoints
cluster_path: features_clustered_010.npy
data_type: 32
dataroot: /home/dataset/
densepose_nc: 3
display_winsize: 512
engine: None
export_onnx: None
feat_num: 30
fineSize: 512
fp16: False
gpu_ids: [0]
how_many: 50
input_nc: 20
instance_feat: False
isTrain: False
label_feat: False
label_nc: 20
loadSize: 1024
load_features: False
local_rank: 0
max_dataset_size: inf
model: ov_pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_local_enhancers: 1
name: vfr
nef: 16
netG: global
ngf: 64
niter_fix_global: 0
no_flip: False
no_instance: False
norm: instance
ntest: inf
onnx: None
output_nc: 20
phase: test
resize_or_crop: scale_width
results_dir: ./results/
serial_batches: False
shape_generation: False
tf_log: False
use_dropout: False
use_encoded_image: False
use_generator_last_activation: False
verbose: False

Namespace(appearance_generation=False, aspect_ratio=1.0, batchSize=1, checkpoints_dir='/workspace/home/aden/work/project/o-viton/checkpoints', cluster_path='features_clustered_010.npy', data_type=32, dataroot='/workspace/home/aden/work/project/o-viton/dataset/', densepose_nc=3, display_winsize=512, engine=None, export_onnx=None, feat_num=30, fineSize=512, fp16=False, gpu_ids=[0], how_many=50, input_nc=20, instance_feat=False, isTrain=False, label_feat=True, label_nc=20, loadSize=1024, load_features=False, local_rank=0, max_dataset_size=inf, model='ov_pix2pixHD', nThreads=1, n_blocks_global=9, n_blocks_local=3, n_clusters=10, n_downsample_E=4, n_downsample_global=4, n_local_enhancers=1, name='vfr_shape', nef=16, netG='global', ngf=64, niter_fix_global=0, no_flip=True, no_instance=False, norm='instance', ntest=inf, onnx=None, output_nc=20, phase='test', resize_or_crop='scale_width', results_dir='/workspace/home/aden/work/project/o-viton/results', serial_batches=True, shape_generation=Tru

In [2]:
# data_loader = CreateDataLoader(opt)
# dataset = data_loader.load_data()
augment = {}
augment['1'] = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), scale=(0.8, 1.2), shear=20),
        transforms.ToTensor(),
        transforms.Normalize((0.5, ), (0.5, ))])  # change to [C, H, W]

augment['2'] = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5, ), (0.5, ))])  # change to [C, H, W]

test_dataset = TestDataset(opt, augment)

test_dataloader = DataLoader(test_dataset,
                              shuffle=False,
                              drop_last=False,
                              num_workers=6,
                              batch_size=opt.batchSize,
                              pin_memory=True)
dataset_size = len(test_dataset)
print('#testing images = %d' % dataset_size)

## FOR DEBUGGING 
# input_dict = {
#             'query_parse_map': A_tensor_label,
#             'ref_parse_map': B_tensor_label,
#             'query_seg_map': query_label_seg_mask,
#             'ref_seg_map': ref_label_seg_mask,
#             'query_img': A_tensor_img,
#             'ref_img': B_tensor_img,
#             'C_tensor': C_tensor
#         }
# for i in test_dataset[0].keys():
#     try:
#         print('{}-------'.format(i),test_dataset[0][i].shape)
#     except Exception as e:
#         print(e)

#testing images = 1016


In [4]:
model = create_model(opt)

--------- model used --------- ov_pix2pixHD
GlobalGenerator(
  (model): Sequential(
    (0): ReflectionPad2d((3, 3, 3, 3))
    (1): Conv2d(227, 64, kernel_size=(7, 7), stride=(1, 1))
    (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (3): ReLU(inplace=True)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (11): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 1024, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (14): Inst

/workspace/home/aden/work/project/o-viton/checkpoints/vfr_shape/latest_net_E.pth


In [ ]:
# torch.unique(test_dataset[0]['C_tensor'])

In [ ]:
# from matplotlib import pyplot as plt
# from PIL import Image
# from PIL import Image
# import numpy as np
# x = test_dataset[0]['C_tensor'].cpu().float().numpy()
# for i in range(20):
#     plt.show()
#     plt.imshow(Image.fromarray((x[i,:,:]).astype(np.uint8)))

In [6]:

visualizer = Visualizer(opt)
# create website
web_dir = os.path.join(opt.results_dir, opt.name, '%s_%s' % (opt.phase, opt.which_epoch))
webpage = html.HTML(web_dir, 'Experiment = %s, Phase = %s, Epoch = %s' % (opt.name, opt.phase, opt.which_epoch))
for i, data in enumerate(test_dataloader):
    #if i >= opt.how_many:
    #    break
    query_ref_mixed, generated = model.inference_forward_shape(data['query_parse_map'],data['ref_parse_map'],
                                                              data['C_tensor'])
    visuals = OrderedDict([('query', util.tensor2label(data['query_parse_map'][0], opt.label_nc)),
                           ('ref', util.tensor2label(data['ref_parse_map'][0], opt.label_nc)),
                           ('query_ref_mixed', util.tensor2label(query_ref_mixed.data[0], opt.label_nc)),
                           ('synthesized_Simage', util.tensor2label(generated.data[0], opt.label_nc)),
                          ('synthesized_image_edgemap', util.tensor2edgemap(torch.softmax(generated.data[0],dim=0)))])
    img_path = data['path']
    print('process image... %s' % img_path)
    visualizer.save_images(webpage, visuals, img_path)

webpage.save()

process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/009809_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/009822_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/009832_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/009843_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/009852_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/009860_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/009872_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/009882_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/009894_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/00990

process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/010599_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/010608_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/010620_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/010628_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/010636_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/010644_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/010653_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/010662_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/010670_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/01068

process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/011381_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/011392_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/011401_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/011415_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/011425_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/011436_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/011447_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/011457_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/011466_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/01147

process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/012158_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/012166_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/012174_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/012183_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/012193_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/012202_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/012219_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/012228_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/012236_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/01224

process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/012944_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/012953_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/012964_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/012975_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/012987_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/012996_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/013007_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/013019_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/013028_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/01303

...

process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/016121_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/016130_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/016145_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/016154_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/016163_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/016172_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/016183_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/016191_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/016206_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/01622

process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/016943_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/016951_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/016962_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/016971_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/016981_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/016992_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/017002_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/017012_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/017021_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/01703

process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/017740_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/017750_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/017761_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/017770_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/017779_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/017788_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/017796_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/017805_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/017814_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/01782

process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/018515_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/018524_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/018532_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/018544_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/018552_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/018561_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/018569_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/018580_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/018590_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/01859

process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/019302_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/019310_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/019319_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/019332_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/019340_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/019350_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/019360_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/019368_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/019376_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/01938

In [ ]:
# import numpy as np
# np.unique(visuals['synthesized_image_edgemap'])

In [ ]:
# from PIL import Image
# test = Image.open('./datasets/fashion_compatibility/test_query_ref_label/0_N00098_synthesized_image_edgemap.jpg')
# print(np.unique(np.array(test)))
# test = Image.open('./results/fashion_compatibility_shape/test_latest/images/0_N00098_synthesized_image_edgemap.png')
# np.unique(np.array(test))

In [11]:
import os
from collections import OrderedDict
from torch.autograd import Variable
from options.test_options import TestOptions
from data.ov_test_dataset import TestDataset
from models.models import create_model
import util.util as util
from util.visualizer import Visualizer
from util import html
import torch
from torchvision import transforms
from torch.utils.data import DataLoader

opt = TestOptions().parse(save=False)
opt.nThreads = 1   # test code only supports nThreads = 1
opt.batchSize = 1  # test code only supports batchSize = 1
opt.serial_batches = True  # no shuffle
opt.no_flip = True  # no flip

opt.label_feat = True
opt.model = 'ov_pix2pixHD'
opt.name = 'vfr_shape'
# opt.dataroot= './datasets/fashion_compatibility/'
opt.appearance_generation = True
opt.use_generator_last_activation = True
opt.input_nc= 20 
opt.output_nc = 3

opt.dataroot= '/workspace/home/aden/work/project/o-viton/dataset/'
opt.checkpoints_dir = "/workspace/home/aden/work/project/o-viton/checkpoints"
opt.results_dir  = "/workspace/home/aden/work/project/o-viton/results"


------------ Options -------------
appearance_generation: False
aspect_ratio: 1.0
batchSize: 1
checkpoints_dir: ./checkpoints
cluster_path: features_clustered_010.npy
data_type: 32
dataroot: /home/dataset/
densepose_nc: 3
display_winsize: 512
engine: None
export_onnx: None
feat_num: 30
fineSize: 512
fp16: False
gpu_ids: [0]
how_many: 50
input_nc: 20
instance_feat: False
isTrain: False
label_feat: False
label_nc: 20
loadSize: 1024
load_features: False
local_rank: 0
max_dataset_size: inf
model: ov_pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_local_enhancers: 1
name: vfr
nef: 16
netG: global
ngf: 64
niter_fix_global: 0
no_flip: False
no_instance: False
norm: instance
ntest: inf
onnx: None
output_nc: 20
phase: test
resize_or_crop: scale_width
results_dir: ./results/
serial_batches: False
shape_generation: False
tf_log: False
use_dropout: False
use_encoded_image: False
use_generator_last_activation: False
verbose: False

In [12]:
# data_loader = CreateDataLoader(opt)
# dataset = data_loader.load_data()
augment = {}
augment['1'] = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), scale=(0.8, 1.2), shear=20),
        transforms.ToTensor(),
        transforms.Normalize((0.5, ), (0.5, ))])  # change to [C, H, W]

augment['2'] = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5, ), (0.5, ))])  # change to [C, H, W]

test_dataset = TestDataset(opt, augment)

test_dataloader = DataLoader(test_dataset,
                              shuffle=False,
                              drop_last=False,
                              num_workers=6,
                              batch_size=opt.batchSize,
                              pin_memory=True)
dataset_size = len(test_dataset)
print('#testing images = %d' % dataset_size)

# # FOR DEBUGGING 
# input_dict = {
#             'query_parse_map': A_tensor_label,
#             'ref_parse_map': B_tensor_label,
#             'query_seg_map': query_label_seg_mask,
#             'ref_seg_map': ref_label_seg_mask,
#             'query_img': A_tensor_img,
#             'ref_img': B_tensor_img,
#             'C_tensor': C_tensor
#         }
# for i in test_dataset[0].keys():
#     try:
#         print('{}-------'.format(i),test_dataset[0][i].shape)
#     except Exception as e:
#         print(e)

#testing images = 1016


In [13]:
model = create_model(opt)

--------- model used --------- ov_pix2pixHD
GlobalGenerator(
  (model): Sequential(
    (0): ReflectionPad2d((3, 3, 3, 3))
    (1): Conv2d(50, 64, kernel_size=(7, 7), stride=(1, 1))
    (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (3): ReLU(inplace=True)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (11): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 1024, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (14): Insta

Pretrained network G has fewer layers; The following are not initialized:
['model']
/workspace/home/aden/work/project/o-viton/checkpoints/vfr_shape/latest_net_E.pth
Pretrained network E has fewer layers; The following are not initialized:
['model']


In [15]:
visualizer = Visualizer(opt)
# create website
web_dir = os.path.join(opt.results_dir, opt.name, '%s_%s' % (opt.phase, opt.which_epoch))
webpage = html.HTML(web_dir, 'Experiment = %s, Phase = %s, Epoch = %s' % (opt.name, opt.phase, opt.which_epoch))
for i, data in enumerate(test_dataloader):
    
    generated = model.inference_forward_appearance(data['query_img'],data['query_parse_map'],
                                                              data['query_seg_map'],data['ref_img'],
                                                              data['ref_parse_map'],data['ref_seg_map'],
                                                              data['C_tensor'])
    
    visuals = OrderedDict([('query_img', util.tensor2im(data['query_img'][0])),
                           ('ref_image', util.tensor2im(data['ref_img'][0])),
                           ('generated_parse_map', util.tensor2label(data['C_tensor'][0], opt.label_nc)),
                           ('synthesized_image', util.tensor2im(generated.data[0]))])
    
    img_path = data['path']
    print('process image... %s' % img_path)
    visualizer.save_images(webpage, visuals, img_path)

webpage.save()

process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/009809_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/009822_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/009832_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/009843_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/009852_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/009860_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/009872_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/009882_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/009894_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/00990

process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/010599_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/010608_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/010620_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/010628_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/010636_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/010644_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/010653_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/010662_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/010670_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/01068

process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/011381_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/011392_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/011401_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/011415_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/011425_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/011436_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/011447_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/011457_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/011466_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/01147

process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/012158_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/012166_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/012174_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/012183_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/012193_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/012202_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/012219_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/012228_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/012236_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/01224

process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/012944_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/012953_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/012964_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/012975_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/012987_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/012996_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/013007_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/013019_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/013028_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/01303

...

process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/016121_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/016130_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/016145_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/016154_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/016163_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/016172_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/016183_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/016191_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/016206_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/01622

process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/016943_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/016951_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/016962_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/016971_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/016981_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/016992_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/017002_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/017012_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/017021_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/01703

process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/017740_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/017750_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/017761_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/017770_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/017779_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/017788_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/017796_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/017805_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/017814_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/01782

process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/018515_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/018524_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/018532_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/018544_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/018552_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/018561_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/018569_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/018580_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/018590_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/01859

process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/019302_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/019310_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/019319_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/019332_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/019340_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/019350_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/019360_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/019368_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/019376_0.png']
process image... ['/workspace/home/aden/work/project/o-viton/dataset/test_query_label/01938

In [ ]:
#online

In [1]:
import torch
from torchvision import transforms
from torch.utils.data import DataLoader

In [2]:
app_feature_vectors_query = torch.zeros((20,30)).float().cuda()

In [3]:
app_feature_vectors_query[0,0] = 1

In [4]:
import os
from collections import OrderedDict
from torch.autograd import Variable
from options.test_options import TestOptions
from data.ov_test_dataset import TestDataset
from models.models import create_model
import util.util as util
from util.visualizer import Visualizer
from util import html
import torch
from torchvision import transforms
from torch.utils.data import DataLoader

opt = TestOptions().parse(save=False)
opt.nThreads = 1   # test code only supports nThreads = 1
opt.batchSize = 1  # test code only supports batchSize = 1
opt.serial_batches = True  # no shuffle
opt.no_flip = True  # no flip

opt.model = 'ov_pix2pixHD'
opt.name = 'vfr'
# opt.dataroot= './datasets/fashion_compatibility/'
opt.appearance_generation = True
opt.use_generator_last_activation = True
opt.input_nc= 20 
opt.output_nc = 3

opt.dataroot= '/workspace/home/aden/work/project/o-viton/online-dataset/'
opt.checkpoints_dir = "/workspace/home/aden/work/project/o-viton/vfr_online"
opt.web_dir = "/workspace/home/aden/work/project/o-viton/online-dataset/results"
opt.load_pretrain = "/workspace/home/aden/work/project/o-viton/checkpoints/vfr_online"
opt.results_dir  = "/workspace/home/aden/work/project/o-viton/online-dataset/results"

opt

------------ Options -------------
appearance_generation: False
aspect_ratio: 1.0
batchSize: 1
checkpoints_dir: ./checkpoints
cluster_path: features_clustered_010.npy
data_type: 32
dataroot: /home/dataset/
densepose_nc: 3
display_winsize: 512
engine: None
export_onnx: None
feat_num: 30
fineSize: 512
fp16: False
gpu_ids: [0]
how_many: 50
input_nc: 20
instance_feat: False
isTrain: False
label_feat: False
label_nc: 20
loadSize: 1024
load_features: False
local_rank: 0
max_dataset_size: inf
model: ov_pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_local_enhancers: 1
name: vfr
nef: 16
netG: global
ngf: 64
niter_fix_global: 0
no_flip: False
no_instance: False
norm: instance
ntest: inf
onnx: None
output_nc: 20
phase: test
resize_or_crop: scale_width
results_dir: ./results/
serial_batches: False
shape_generation: False
tf_log: False
use_dropout: False
use_encoded_image: False
use_generator_last_activation: False
verbose: False

Namespace(appearance_generation=True, aspect_ratio=1.0, batchSize=1, checkpoints_dir='/workspace/home/aden/work/project/o-viton/vfr_online', cluster_path='features_clustered_010.npy', data_type=32, dataroot='/workspace/home/aden/work/project/o-viton/online-dataset/', densepose_nc=3, display_winsize=512, engine=None, export_onnx=None, feat_num=30, fineSize=512, fp16=False, gpu_ids=[0], how_many=50, input_nc=20, instance_feat=False, isTrain=False, label_feat=False, label_nc=20, loadSize=1024, load_features=False, load_pretrain='/workspace/home/aden/work/project/o-viton/checkpoints/vfr_online', local_rank=0, max_dataset_size=inf, model='ov_pix2pixHD', nThreads=1, n_blocks_global=9, n_blocks_local=3, n_clusters=10, n_downsample_E=4, n_downsample_global=4, n_local_enhancers=1, name='vfr', nef=16, netG='global', ngf=64, niter_fix_global=0, no_flip=True, no_instance=False, norm='instance', ntest=inf, onnx=None, output_nc=3, phase='test', resize_or_crop='scale_width', results_dir='/workspace/h

In [5]:
# data_loader = CreateDataLoader(opt)
# dataset = data_loader.load_data()
augment = {}
augment['1'] = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), scale=(0.8, 1.2), shear=20),
        transforms.ToTensor(),
        transforms.Normalize((0.5, ), (0.5, ))])  # change to [C, H, W]

augment['2'] = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5, ), (0.5, ))])  # change to [C, H, W]

test_dataset = TestDataset(opt, augment)

test_dataloader = DataLoader(test_dataset,
                              shuffle=False,
                              drop_last=False,
                              num_workers=1,
                              batch_size=opt.batchSize,
                              pin_memory=True)
dataset_size = len(test_dataset)
print('#testing images = %d' % dataset_size)

# # FOR DEBUGGING 
# input_dict = {
#             'query_parse_map': A_tensor_label,
#             'ref_parse_map': B_tensor_label,
#             'query_seg_map': query_label_seg_mask,
#             'ref_seg_map': ref_label_seg_mask,
#             'query_img': A_tensor_img,
#             'ref_img': B_tensor_img,
#             'C_tensor': C_tensor
#         }
# for i in test_dataset[0].keys():
#     try:
#         print('{}-------'.format(i),test_dataset[0][i].shape)
#     except Exception as e:
#         print(e)

#testing images = 10


In [9]:
import os
from collections import OrderedDict
from torch.autograd import Variable
from options.train_options import TrainOptions
from data.ov_test_dataset import TestDataset
from models.models import create_model
import util.util as util
from util.visualizer import Visualizer
from util import html
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
import time

opt = TrainOptions().parse(save=False)
opt.nThreads = 1   # test code only supports nThreads = 1
opt.batchSize = 1  # test code only supports batchSize = 1
opt.serial_batches = True  # no shuffle
opt.no_flip = True  # no flip

opt.model = 'ov_pix2pixHD_online'
opt.name = 'vfr_online'

# opt.dataroot= './datasets/fashion_compatibility/'
opt.appearance_generation = True
opt.use_generator_last_activation = True
opt.input_nc= 20 
opt.output_nc = 3

opt.dataroot= '/workspace/home/aden/work/project/o-viton/online-dataset/'
opt.checkpoints_dir = "/workspace/home/aden/work/project/o-viton/vfr_online"
opt.web_dir = "/workspace/home/aden/work/project/o-viton/online-dataset/results"
opt.load_pretrain = "/workspace/home/aden/work/project/o-viton/vfr_online/vfr"
opt.results_dir  = "/workspace/home/aden/work/project/o-viton/online-dataset/results"

opt

------------ Options -------------
appearance_generation: False
batchSize: 1
beta1: 0.5
checkpoints_dir: ./checkpoints
continue_train: False
data_type: 32
dataroot: /home/dataset/
debug: False
densepose_nc: 3
display_freq: 100
display_winsize: 512
feat_num: 30
fineSize: 512
fp16: False
gpu_ids: [0]
input_nc: 20
instance_feat: False
isTrain: True
label_feat: False
label_nc: 20
lambda_feat: 10.0
loadSize: 1024
load_features: False
load_pretrain: 
local_rank: 0
lr: 0.001
max_dataset_size: inf
model: ov_pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_layers_D: 3
n_local_enhancers: 1
name: vfr
ndf: 64
nef: 16
netG: global
ngf: 64
niter: 100
niter_decay: 100
niter_fix_global: 0
no_ce_loss: False
no_flip: False
no_ganFeat_loss: False
no_html: False
no_instance: False
no_lsgan: False
no_vgg_loss: False
norm: instance
num_D: 2
output_nc: 20
phase: train
pool_size: 0
print_freq: 100
resize_or_crop: scale_width
save_epoch_freq: 

Namespace(appearance_generation=True, batchSize=1, beta1=0.5, checkpoints_dir='/workspace/home/aden/work/project/o-viton/vfr_online', continue_train=False, data_type=32, dataroot='/workspace/home/aden/work/project/o-viton/online-dataset/', debug=False, densepose_nc=3, display_freq=100, display_winsize=512, feat_num=30, fineSize=512, fp16=False, gpu_ids=[0], input_nc=20, instance_feat=False, isTrain=True, label_feat=False, label_nc=20, lambda_feat=10.0, loadSize=1024, load_features=False, load_pretrain='/workspace/home/aden/work/project/o-viton/vfr_online/vfr', local_rank=0, lr=0.001, max_dataset_size=inf, model='ov_pix2pixHD_online', nThreads=1, n_blocks_global=9, n_blocks_local=3, n_clusters=10, n_downsample_E=4, n_downsample_global=4, n_layers_D=3, n_local_enhancers=1, name='vfr_online', ndf=64, nef=16, netG='global', ngf=64, niter=100, niter_decay=100, niter_fix_global=0, no_ce_loss=False, no_flip=True, no_ganFeat_loss=False, no_html=False, no_instance=False, no_lsgan=False, no_vgg_

In [10]:
model = create_model(opt)

--------- model used --------- ov_pix2pixHD_online
GlobalGenerator(
  (model): Sequential(
    (0): ReflectionPad2d((3, 3, 3, 3))
    (1): Conv2d(50, 64, kernel_size=(7, 7), stride=(1, 1))
    (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (3): ReLU(inplace=True)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (11): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 1024, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (14)

Encoder(
  (model): Sequential(
    (0): ReflectionPad2d((3, 3, 3, 3))
    (1): Conv2d(23, 16, kernel_size=(7, 7), stride=(1, 1))
    (2): InstanceNorm2d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (3): ReLU(inplace=True)
    (4): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (5): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (6): ReLU(inplace=True)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (9): ReLU(inplace=True)
    (10): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (11): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (12): ReLU(inplace=True)
    (13): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (14): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track

In [11]:
# Training Visualizer
visualizer = Visualizer(opt)

# Optimizers
optimizer_G, optimizer_D = model.module.optimizer_G, model.module.optimizer_D

create web directory /workspace/home/aden/work/project/o-viton/vfr_online/vfr_online/web...


In [12]:
prev_j = 0
index_check = 10
for i, data in enumerate(test_dataloader, start=0):
    for j in range(0,100):
        save_fake = True if j % index_check == 0 else False
        losses, generated = model(data['query_img'],data['query_parse_map'],
                                                    data['query_seg_map'],data['ref_img'],
                                                    data['ref_parse_map'],data['ref_seg_map'],
                                                    data['C_tensor'],infer=save_fake)

        # sum per device losses
        losses = [torch.mean(x) if not isinstance(x, int) else x for x in losses]
        loss_dict = dict(zip(model.module.loss_names, losses))

        # calculate final loss scalar
        loss_ref = (loss_dict['G_GAN_REF'] + loss_dict['G_VGG'])
        loss_query = loss_dict['G_GAN_QUERY']
        loss_online = loss_ref + loss_query

        ############### Backward Pass ####################
        # update generator weights
        optimizer_G.zero_grad()
        loss_online.backward()
        optimizer_G.step()
        
        print(f'{i}th image, {j}th step, loss_ref {loss_ref}, loss_query {loss_query}, loss_online {loss_online}')
        
        ############## Display results and errors ##########
        if save_fake:
            visuals = OrderedDict([('query_img', util.tensor2im(data['query_img'][0])),
                               ('ref_image', util.tensor2im(data['ref_img'][0])),
                               ('synthesized_image', util.tensor2im(generated.data[0]))])
            visualizer.display_current_results(visuals, (j // index_check) + prev_j, i)
    prev_j = (j // index_check)*(i+1)

/workspace/home/aden/work/github/o-viton/inference/models/ov_pix2pixHD_model_online.py:200: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  indices = (selected_seg_mask_tensor == int(


0th image, 0th step, loss_ref 13.221083641052246, loss_query 4.012631416320801, loss_online 17.233715057373047


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 5.79 GiB total capacity; 4.63 GiB already allocated; 12.94 MiB free; 4.65 GiB reserved in total by PyTorch)

In [ ]:
# Test exercises

In [ ]:
import numpy as np
from PIL import Image

In [ ]:
image_numpy = test_dataset[2]['ref_img'].cpu().float().numpy()
image_numpy = (np.transpose(image_numpy, (1, 2, 0)) + 1) / 2.0 * 255.0
image_numpy = image_numpy.astype(np.uint8)
Image.fromarray(image_numpy)

In [ ]:
org_seg_map = test_dataset[2]['ref_parse_map'].clone()

# unnormalize it 
unnormalize_seg_map = org_seg_map.cpu().float().numpy()
unnormalize_seg_map = (np.transpose(unnormalize_seg_map, (1, 2, 0)) + 1) / 2.0
unnormalize_seg_map = unnormalize_seg_map.astype(np.uint8)
np.unique(unnormalize_seg_map)

# filter the top section
filter_part = unnormalize_seg_map[:, :, 5] + unnormalize_seg_map[:, :, 6] + unnormalize_seg_map[ :, :,7]  # 5,6,7,
filter_part = filter_part > 0
filter_part = np.expand_dims(filter_part,-1)


image_numpy = test_dataset[2]['ref_img'].cpu().float().numpy()
image_numpy = (np.transpose(image_numpy, (1, 2, 0)) + 1) / 2.0 * 255.0
image_numpy = image_numpy.astype(np.uint8)
source_img = image_numpy * filter_part # other part normalized 0 --> 127 gray
Image.fromarray(source_img)

In [ ]:
ref_parse_map_online = torch.zeros((20, 512, 256)).float().cuda()
for num_seg_channel in range(20):
    if 4 < num_seg_channel < 8:
        ref_parse_map_online[num_seg_channel,:,:] = test_dataset[2]['ref_parse_map'][num_seg_channel,:,:]

In [ ]:
ref_parse_map_online = (ref_parse_map_online + 1)/2
filter_part = ref_parse_map_online[5, :, :] + ref_parse_map_online[6, :, :] + ref_parse_map_online[7, :, :]  # 5,6,7,
filter_part = filter_part > 0
filter_part = torch.unsqueeze(filter_part,0).float().cuda()
src = test_dataset[2]['ref_img'].float().cuda()
src = (src + 1)/2
source_img = src * filter_part # other part normalized 0 --> 127 gray
test = source_img.clone()
image_numpy = source_img.cpu().float().numpy()
image_numpy = np.transpose(image_numpy,(1, 2, 0))* 255.0
image_numpy = image_numpy.astype(np.uint8)
Image.fromarray(image_numpy)

In [ ]:
norm = transforms.Normalize((0.5,), (0.5,))
out = norm(test)
torch.unique(out)